# 配当利回り

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/J-Quants/jquants-api-client-python/blob/master/examples/20220825-003-dividend.ipynb)


このノートブックでは J-Quants APIから取得できるデータから配当性向、配当利回り、および直近1,3ヶ月のリターンを一覧で表示します。


----

**このノートブックはGoogle Driveを使用します。**

- Google Drive の以下のファイルにリフレッシュトークンが書き込まれていることを想定しています。
    - `MyDrive/drive_ws/secret/jquantsapi-key.txt`
- Google Drive の以下のフォルダーにデータを書き込みます。
    - `MyDrive/drive_ws/marketdata`

In [1]:
# python version　3｡10｡9で動作確認
!python --version

Python 3.10.9


In [2]:
# 必要なモジュールをインストールします。
! python -m pip install jquants-api-client japanize-matplotlib

In [3]:
import os
from datetime import datetime

import japanize_matplotlib
import jquantsapi
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [4]:
# pandas の表示制限を調整します
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.width", 2000)

In [5]:
# プロット用の設定をします
sns.set(rc={'figure.figsize': (15, 10)})
sns.set(font_scale=1.5)
sns.set_style('whitegrid')

japanize_matplotlib.japanize()

In [6]:
# 一度取得したデータは Google Drive 上に保存して再利用します。
# 保存先ディレクトリを指定します。
STORAGE_DIR_PATH = "../drive_ws/marketdata"
os.makedirs(STORAGE_DIR_PATH, exist_ok=True)
STORAGE_DIR_PATH

'../drive_ws/marketdata'

In [7]:
# J-Quants APIのトークンを保存してあるファイルを指定します
REFRESH_TOKEN_FILE_PATH = "../drive_ws/secret/jquantsapi-key.txt"

In [8]:
def get_refresh_token(refresh_token_file_path: str = REFRESH_TOKEN_FILE_PATH):
    with open(refresh_token_file_path, "r") as f:
        refresh_token = f.read()
    return refresh_token.rstrip().lstrip()

In [9]:
# ファイルからリフレッシュトークンを読み込みます
refresh_token = get_refresh_token()


In [10]:
# J-Quants APIクライアントを初期化します
jqapi = jquantsapi.Client(refresh_token=refresh_token)

In [11]:
# 銘柄情報を取得します
## 原罪時間を取り込み
now = pd.Timestamp.now(tz="Asia/Tokyo")
## 時間が22時より前
if now.hour < 22:
    # データ更新時間前の場合は日付を1日ずらします。
    now -= pd.Timedelta(1, unit="D")

list_file = f"{STORAGE_DIR_PATH}/list_{now.strftime('%Y%m%d')}.csv.gz"
if not os.path.isfile(list_file):
    df_list = jqapi.get_list()
    df_list.to_csv(list_file, compression="gzip", index=False)
    print(f"save file: {list_file}")

# ファイルからデータを読み込みます
print(f"file exists: {list_file}, loading")
df_list = pd.read_csv(list_file, dtype="str")
df_list

file exists: ../drive_ws/marketdata/list_20230211.csv.gz, loading


,Date,Code,CompanyName,Sector17Code,Sector17CodeName,Sector33Code,Sector33CodeName,ScaleCategory,MarketCode,MarketCodeName,Sector17CodeNameEnglish,Sector33CodeNameEnglish,MarketCodeNameEnglish
0,2023-02-13,13010,極洋,1,食品,0050,水産・農林業,TOPIX Small 2,111,プライム,FOODS,"Fishery, Agriculture & Forestry",Prime
1,2023-02-13,13050,ダイワ上場投信−トピックス,99,その他,9999,その他,-,109,その他,OTHER,Other,Others
2,2023-02-13,13060,ＮＥＸＴＦＵＮＤＳＴＯＰＩＸ連動型上場投信,99,その他,9999,その他,-,109,その他,OTHER,Other,Others
3,2023-02-13,13080,上場インデックスファンドＴＯＰＩＸ,99,その他,9999,その他,-,109,その他,OTHER,Other,Others
4,2023-02-13,13090,ＮＥＸＴＦＵＮＤＳＣｈｉｎａＡＭＣ・中国株式・上証５０連動型上場投信,99,その他,9999,その他,-,109,その他,OTHER,Other,Others
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4242,2023-02-13,99930,ヤマザワ,14,小売,6100,小売業,TOPIX Small 2,112,スタンダード,RETAIL TRADE,Retail Trade,Standard
4243,2023-02-13,99940,やまや,14,小売,6100,小売業,TOPIX Small 2,112,スタンダード,RETAIL TRADE,Retail Trade,Standard
4244,2023-02-13,99950,グローセル,13,商社・卸売,6050,卸売業,TOPIX Small 2,111,プライム,COMMERCIAL & WHOLESALE TRADE,Wholesale Trade,Prime
4245,2023-02-13,99960,サトー商会,13,商社・卸売,6050,卸売業,-,112,スタンダード,COMMERCIAL & WHOLESALE TRADE,Wholesale Trade,Standard


##　株価情報の取得

In [12]:
# 株価情報を取得します
now = pd.Timestamp.now(tz="Asia/Tokyo")
# 過去3ヶ月のデータを取得
start_dt = now - pd.Timedelta(95, unit="D")
end_dt = now
if end_dt.hour < 19:
    # データ更新時間前の場合は日付を1日ずらします。
    end_dt -= pd.Timedelta(1, unit="D")
    
price_file = f"{STORAGE_DIR_PATH}/price_{start_dt.strftime('%Y%m%d')}_{end_dt.strftime('%Y%m%d')}.csv.gz"
if not os.path.isfile(price_file):
    df_p = jqapi.get_price_range(start_dt=start_dt, end_dt=end_dt)
    df_p.to_csv(price_file, compression="gzip", index=False)
    df_p.reset_index(drop=True, inplace=True)
    print(f"save file: {price_file}")

# データを読み込みます
print(f"file exists: {price_file}, loading")
df_p = pd.read_csv(price_file, dtype="str")
df_p.reset_index(drop=True, inplace=True)

# 各列のデータ型を調整します
df_p.loc[:, "Date"] = pd.to_datetime(df_p["Date"], format="%Y-%m-%d")
df_p.loc[:, "Open"] = df_p["Open"].astype(np.float64)
df_p.loc[:, "High"] = df_p["High"].astype(np.float64)
df_p.loc[:, "Low"] = df_p["Low"].astype(np.float64)
df_p.loc[:, "Close"] = df_p["Close"].astype(np.float64)
df_p.loc[:, "Volume"] = df_p["Volume"].astype(np.float64)
df_p.loc[:, "TurnoverValue"] = df_p["TurnoverValue"].astype(np.float64)
df_p.loc[:, "AdjustmentFactor"] = df_p["AdjustmentFactor"].astype(np.float64)
df_p.loc[:, "AdjustmentOpen"] = df_p["AdjustmentOpen"].astype(np.float64)
df_p.loc[:, "AdjustmentHigh"] = df_p["AdjustmentHigh"].astype(np.float64)
df_p.loc[:, "AdjustmentLow"] = df_p["AdjustmentLow"].astype(np.float64)
df_p.loc[:, "AdjustmentClose"] = df_p["AdjustmentClose"].astype(np.float64)
df_p.loc[:, "AdjustmentVolume"] = df_p["AdjustmentVolume"].astype(np.float64)

file exists: ../drive_ws/marketdata/price_20221109_20230211.csv.gz, loading


/var/folders/kx/n6w_n8193fn6dy4jwf3fsr6w0000gn/T/ipykernel_2726/2407126456.py:23: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_p.loc[:, "Date"] = pd.to_datetime(df_p["Date"], format="%Y-%m-%d")
/var/folders/kx/n6w_n8193fn6dy4jwf3fsr6w0000gn/T/ipykernel_2726/2407126456.py:24: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_p.loc[:, "Open"] = df_p["Open"].astype(np.float64)
/var/folders/kx/n6w_n8193fn6dy4jwf3fsr6w0000gn/T/ipykernel_2726/2407126456.py:25: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to s

## 財務情報の取得

### コード

In [13]:
# 過去3ヶ月に発表された財務情報を取得します

# 取得する日時設定
now = pd.Timestamp.now(tz="Asia/Tokyo")
# start_dt = now - pd.Timedelta(90, unit="D")

# APIとしてサービスが2017年1月1日からなので最古データから受信
start_dt = pd.Timestamp("2017-01-01", tz="Asia/Tokyo")
end_dt = now
if end_dt.hour < 1:
    # データ更新時間前の場合は日付を1日ずらします。
    end_dt -= pd.Timedelta(1, unit="D")

# ファイル名を決める。既に取得したデータがあればそのデータを使う。
statements_file = f"{STORAGE_DIR_PATH}/statements_{start_dt.strftime('%Y%m%d')}_{end_dt.strftime('%Y%m%d')}.csv.gz"
if not os.path.isfile(statements_file):
    cache_dir = f"{STORAGE_DIR_PATH}/raw_statements"
    os.makedirs(cache_dir, exist_ok=True)
    df_s = jqapi.get_statements_range(
        start_dt=start_dt, end_dt=end_dt, cache_dir=cache_dir
    )
    df_s.to_csv(statements_file, compression="gzip", index=False)
    print(f"save file: {statements_file}")

print(f"file exists: {statements_file}, loading")
df_s = pd.read_csv(statements_file, dtype="str")

# float64にするために"-"をnp.nanに置き換えます
df_s.replace({"－": np.nan}, inplace=True)
df_s["ResultDividendPerShareFiscalYearEnd"] = df_s["ResultDividendPerShareFiscalYearEnd"].astype(np.float64)
df_s["EarningsPerShare"] = df_s["EarningsPerShare"].astype(np.float64)
df_s["ForecastDividendPerShareAnnual"] = df_s["ForecastDividendPerShareAnnual"].astype(np.float64)
df_s["ForecastEarningsPerShare"] = df_s["ForecastEarningsPerShare"].astype(np.float64)

# 日付型に変換します
df_s.loc[:, "DisclosedDate"] = pd.to_datetime(
    df_s["DisclosedDate"], format="%Y-%m-%d"
)
df_s.loc[:, "CurrentPeriodEndDate"] = pd.to_datetime(
    df_s["CurrentPeriodEndDate"], format="%Y-%m-%d"
)
df_s.loc[:, "CurrentFiscalYearStartDate"] = pd.to_datetime(
    df_s["CurrentFiscalYearStartDate"], format="%Y-%m-%d"
)
df_s.loc[:, "CurrentFiscalYearEndDate"] = pd.to_datetime(
    df_s["CurrentFiscalYearEndDate"], format="%Y-%m-%d"
)
df_s.sort_values("DisclosedUnixTime", inplace=True)

file exists: ../drive_ws/marketdata/statements_20170101_20230212.csv.gz, loading


/var/folders/kx/n6w_n8193fn6dy4jwf3fsr6w0000gn/T/ipykernel_2726/589000079.py:36: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_s.loc[:, "DisclosedDate"] = pd.to_datetime(
/var/folders/kx/n6w_n8193fn6dy4jwf3fsr6w0000gn/T/ipykernel_2726/589000079.py:39: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_s.loc[:, "CurrentPeriodEndDate"] = pd.to_datetime(
/var/folders/kx/n6w_n8193fn6dy4jwf3fsr6w0000gn/T/ipykernel_2726/589000079.py:42: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace inst

In [14]:
df_s

,DisclosureNumber,DisclosedDate,ApplyingOfSpecificAccountingOfTheQuarterlyFinancialStatements,AverageNumberOfShares,BookValuePerShare,ChangesBasedOnRevisionsOfAccountingStandard,ChangesInAccountingEstimates,ChangesOtherThanOnesBasedOnRevisionsOfAccountingStandard,CurrentFiscalYearEndDate,CurrentFiscalYearStartDate,CurrentPeriodEndDate,DisclosedTime,DisclosedUnixTime,EarningsPerShare,Equity,EquityToAssetRatio,ForecastDividendPerShare1stQuarter,ForecastDividendPerShare2ndQuarter,ForecastDividendPerShare3rdQuarter,ForecastDividendPerShareAnnual,ForecastDividendPerShareFiscalYearEnd,ForecastEarningsPerShare,ForecastNetSales,ForecastOperatingProfit,ForecastOrdinaryProfit,ForecastProfit,LocalCode,MaterialChangesInSubsidiaries,NetSales,NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock,NumberOfTreasuryStockAtTheEndOfFiscalYear,OperatingProfit,OrdinaryProfit,Profit,ResultDividendPerShare1stQuarter,ResultDividendPerShare2ndQuarter,ResultDividendPerShare3rdQuarter,ResultDividendPerShareAnnual,ResultDividendPerShareFiscalYearEnd,RetrospectiveRestatement,TotalAssets,TypeOfCurrentPeriod,TypeOfDocument
0,20161207453651,2017-01-04,NaN,6848800.0,2671.42,True,False,False,2017-03-31,2016-04-01,2016-12-31,07:30:00,1483482600.0,218.23,18295000000.0,0.817,NaN,NaN,NaN,100.0,50.0,319.76,31800000000,3255000000,3300000000,2190000000,27530,False,22761000000.0,6848800.0,NaN,2147000000.0,2234000000.0,1494000000.0,NaN,50.0,NaN,NaN,NaN,False,22386000000.0,3Q,3QFinancialStatements_Consolidated_JP
1,20161229465838,2017-01-04,NaN,NaN,NaN,NaN,NaN,NaN,2017-06-30,2016-07-01,2016-12-31,12:00:00,1483498800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-93.11,110000000,-465000000,-466000000,-467000000,45750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2Q,ForecastRevision
2,20170104467704,2017-01-04,NaN,1916083.0,NaN,True,False,False,2017-02-28,2016-03-01,2016-11-30,15:00:00,1483509600.0,328.57,7566000000.0,0.301,NaN,NaN,NaN,72.0,36.0,485.36,30200000000,1350000000,1300000000,930000000,33530,False,22128000000.0,2035000.0,118917,820000000.0,778000000.0,629000000.0,NaN,36.0,NaN,NaN,NaN,False,25100000000.0,3Q,3QFinancialStatements_Consolidated_JP
3,20170104467943,2017-01-05,NaN,39636693.0,795.65,True,False,False,2017-03-31,2016-04-01,2016-12-31,08:00:00,1483570800.0,75.75,31348000000.0,0.752,NaN,NaN,NaN,26.0,13.0,110.87,21500000000.0,6400000000,6700000000.0,4400000000.0,74630,False,14619000000.0,53812692.0,14413066.0,4018000000.0,4576000000.0,3002000000.0,NaN,13.0,NaN,NaN,NaN,False,41676000000.0,3Q,3QFinancialStatements_Consolidated_JP
4,20170104467675,2017-01-05,False,11260044.0,NaN,True,False,False,2017-02-28,2016-03-01,2016-11-30,15:00:00,1483596000.0,94.10,17678000000.0,0.638,NaN,NaN,NaN,30.0,15.0,133.17,105500000000.0,2650000000,2700000000.0,1500000000.0,99770,NaN,78930000000.0,12500000.0,1223765.0,1786000000.0,1845000000.0,1059000000.0,NaN,15.0,NaN,NaN,NaN,False,27607000000.0,3Q,3QFinancialStatements_NonConsolidated_JP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118067,20230209505226,2023-02-10,NaN,12706503.0,NaN,True,False,False,2023-03-31,2022-04-01,2022-12-31,18:00:00,1676019600.0,-1.91,1479000000.0,0.83,NaN,NaN,NaN,0.0,0.0,2.36,3000000000,30000000,60000000,30000000,48140,False,2098000000,13015222.0,308719,-71000000.0,-15000000.0,-24000000.0,NaN,0.0,NaN,NaN,NaN,False,1782000000.0,3Q,3QFinancialStatements_Consolidated_JP
118068,20230209505287,2023-02-10,NaN,5756447.0,1549.89,True,False,False,2022-12-31,2022-01-01,2022-12-31,18:00:00,1676019600.0,220.99,8918000000.0,0.751,NaN,NaN,NaN,35.0,35.0,121.65,9500000000,1000000000,1000000000,700000000,39480,NaN,11994000000,5815294.0,60988,1968000000.0,1975000000.0,1272000000.0,NaN,NaN,NaN,50.0,50.0,False,11880000000.0,FY,FYFinancialStatements_NonConsolidated_JP
118069,20230210506904,2023-02-10,NaN,2769745.0,NaN,True,False,False,2023-03-31,2022-04-01,2022-12-31,18:00:0

### 情報リスト

DisclosureNumber	適時開示番号（TDnet内で一意になるコード）<br>
DisclosedDate	適時開示された日<br>
ApplyingOfSpecificAccountingOfTheQuarterlyFinancialStatements	四半期財務諸表の作成に特有の会計処理の適用<br>
AverageNumberOfShares	会計期中の平均株式数<br>
BookValuePerShare	一株あたりの純資産<br>
ChangesBasedOnRevisionsOfAccountingStandard	会計基準等の改正に伴う会計方針の変更<br>
ChangesInAccountingEstimates	会計上の見積りの変更<br>
ChangesOtherThanOnesBasedOnRevisionsOfAccountingStandard	会計基準等の改正に伴う変更以外の会計方針の変更<br>
CurrentFiscalYearEndDate	当事業年度終了日<br>
CurrentFiscalYearStartDate	当事業年度開始日<br>
CurrentPeriodEndDate	当会計期間終了日<br>
DisclosedTime	開示時刻<br>
DisclosedUnixTime	開示UnixTime<br>
EarningsPerShare	1株当たり当期純利益<br>
Equity	純資産<br>
EquityToAssetRatio	自己資本比率<br>
ForecastDividendPerShare1stQuarter	第1四半期末の配当予想<br>
ForecastDividendPerShare2ndQuarter	第2四半期末の配当予想<br>
ForecastDividendPerShare3rdQuarter	第3四半期末の配当予想<br>
ForecastDividendPerShareAnnual	通期の配当予想の合計値<br>
ForecastDividendPerShareFiscalYearEnd	期末の配当予想<br>
ForecastEarningsPerShare	1株当たり当期純利益の通期予想<br>
ForecastNetSales	売上高_通期予想<br>
ForecastOperatingProfit	営業利益_通期予想<br>
ForecastOrdinaryProfit	経常利益_通期予想<br>
ForecastProfit	当期純利益_通期予想<br>
LocalCode	株式銘柄コード<br>
MaterialChangesInSubsidiaries	期中における重要な子会社の異動<br>
NetSales	売上高<br>
NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock	期末発行済株式数<br>
NumberOfTreasuryStockAtTheEndOfFiscalYear	期末自己株式数<br>
OperatingProfit	営業利益<br>
OrdinaryProfit	経常利益<br>
Profit	当期純利益<br>
ResultDividendPerShare1stQuarter	第1四半期末の配当実績<br>
ResultDividendPerShare2ndQuarter	第2四半期末の配当実績<br>
ResultDividendPerShare3rdQuarter	第3四半期末の配当実績<br>
ResultDividendPerShareAnnual	通期の配当実績の合計<br>
ResultDividendPerShareFiscalYearEnd	期末の配当実績<br>
RetrospectiveRestatement	修正再表示<br>
TotalAssets	総資産<br>
TypeOfCurrentPeriod	当会計期間の種類<br>
TypeOfDocument	書類種別<br>

## 株価データを作業ワークへ
財務情報　：　df_work = df_s.copy()<br>
株価情報　：　df_p_work = df_p.copy()

In [15]:
# 作業用にデータをコピーします
df_work = df_s.copy()
df_p_work = df_p.copy()

## 前処理

In [27]:
# 財務情報を銘柄ごとに重複を排除して最新の財務情報のみを使用します
df_work.sort_values("DisclosedUnixTime", inplace=True)
df_work = df_work.drop_duplicates(["LocalCode"], keep="last")

# 終値が0の場合は前営業日の終値を使用します
df_p_work.sort_values(["Code", "Date"], inplace=True)
df_p_work["AdjustmentClose"].replace({0.0: np.nan}, inplace=True)
df_p_work.loc[:, "AdjustmentClose"] = df_p_work.groupby("Code")["AdjustmentClose"].ffill()
# 終値がnanの場合は翌営業日の終値を使用します (データの先頭)
df_p_work.loc[:, "AdjustmentClose"] = df_p_work.groupby("Code")["AdjustmentClose"].bfill()
# 各銘柄の直近のリターンを算出します
def _calc_return(df, bdays):
  return (df["AdjustmentClose"].iat[-1] / df["AdjustmentClose"].iloc[-bdays:].iat[0]) - 1
df_p_work.sort_values(["Code", "Date"], inplace=True)
df_returns_1months = df_p_work.groupby("Code").apply(_calc_return, 20).rename("1ヶ月リターン")
df_returns_3months = df_p_work.groupby("Code").apply(_calc_return, 60).rename("3ヶ月リターン")
# リターンと結合します
df_work = pd.merge(df_work, df_returns_1months, left_on=["LocalCode"], right_index=True, how="left")
df_work = pd.merge(df_work, df_returns_3months, left_on=["LocalCode"], right_index=True, how="left")

# 配当利回りを計算するために直近の終値を取得します
df_close = df_p_work.loc[df_p_work["Date"] == df_p_work["Date"].max(), ["Code", "Date", "AdjustmentClose"]]
# 直近の株価と結合します
df_work = pd.merge(df_work, df_close, left_on=["LocalCode"], right_on=["Code"], how="left")
# 配当利回りを算出します
df_work["配当利回り"] = df_work["ResultDividendPerShareFiscalYearEnd"] / df_work["AdjustmentClose"]
# 予想配当利回りを算出します
df_work["予想配当利回り"] = df_work["ForecastDividendPerShareAnnual"] / df_work["AdjustmentClose"]

# 配当性向を算出します
df_work["配当性向"] = df_work["ResultDividendPerShareFiscalYearEnd"] / df_work["EarningsPerShare"] 
# 予想配当性向を算出します
df_work["予想配当性向"] = df_work["ForecastDividendPerShareAnnual"] / df_work["ForecastEarningsPerShare"]

# 銘柄名と結合します
df_work = pd.merge(df_work, df_list, left_on=["LocalCode"], right_on=["Code"])

# 表示用に開示日を追加します
df_work["開示日"] = df_work["DisclosedDate"].dt.strftime("%Y-%m-%d")

/var/folders/kx/n6w_n8193fn6dy4jwf3fsr6w0000gn/T/ipykernel_2726/3478403726.py:18: FutureWarning: Passing 'suffixes' which cause duplicate columns {'1ヶ月リターン_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_work = pd.merge(df_work, df_returns_1months, left_on=["LocalCode"], right_index=True, how="left")
/var/folders/kx/n6w_n8193fn6dy4jwf3fsr6w0000gn/T/ipykernel_2726/3478403726.py:19: FutureWarning: Passing 'suffixes' which cause duplicate columns {'3ヶ月リターン_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_work = pd.merge(df_work, df_returns_3months, left_on=["LocalCode"], right_index=True, how="left")
/var/folders/kx/n6w_n8193fn6dy4jwf3fsr6w0000gn/T/ipykernel_2726/3478403726.py:36: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Date_x', 'Code_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_work = pd.merge(df_work, df_list, left_on=["LocalCode"], right_on=["Cod

In [28]:
# 財務情報を銘柄ごとに重複を排除して最新の財務情報のみを使用します
df_work.sort_values("DisclosedUnixTime", inplace=True)
df_work = df_work.drop_duplicates(["LocalCode"], keep="last")

# 終値が0の場合は前営業日の終値を使用します
df_p_work.sort_values(["Code", "Date"], inplace=True)
df_p_work["AdjustmentClose"].replace({0.0: np.nan}, inplace=True)
df_p_work.loc[:, "AdjustmentClose"] = df_p_work.groupby("Code")["AdjustmentClose"].ffill()
# 終値がnanの場合は翌営業日の終値を使用します (データの先頭)
df_p_work.loc[:, "AdjustmentClose"] = df_p_work.groupby("Code")["AdjustmentClose"].bfill()
# 各銘柄の直近のリターンを算出します
def _calc_return(df, bdays):
  return (df["AdjustmentClose"].iat[-1] / df["AdjustmentClose"].iloc[-bdays:].iat[0]) - 1
df_p_work.sort_values(["Code", "Date"], inplace=True)
df_returns_1months = df_p_work.groupby("Code").apply(_calc_return, 20).rename("1ヶ月リターン")
df_returns_3months = df_p_work.groupby("Code").apply(_calc_return, 60).rename("3ヶ月リターン")
# リターンと結合します
df_work = pd.merge(df_work, df_returns_1months, left_on=["LocalCode"], right_index=True, how="left")
df_work = pd.merge(df_work, df_returns_3months, left_on=["LocalCode"], right_index=True, how="left")

# 配当利回りを計算するために直近の終値を取得します
df_close = df_p_work.loc[df_p_work["Date"] == df_p_work["Date"].max(), ["Code", "Date", "AdjustmentClose"]]
# 直近の株価と結合します
df_work = pd.merge(df_work, df_close, left_on=["LocalCode"], right_on=["Code"], how="left")
# 配当利回りを算出します
df_work["配当利回り"] = df_work["ResultDividendPerShareFiscalYearEnd"] / df_work["AdjustmentClose"]
# 予想配当利回りを算出します
df_work["予想配当利回り"] = df_work["ForecastDividendPerShareAnnual"] / df_work["AdjustmentClose"]

# 実績PER：株価を1株当たりの利益で割って求めます。PER（倍） ＝ 株価 ÷ 1株当たり当期純利益
df_work["PER"] = df_work["AdjustmentClose"] / df_work["EarningsPerShare"]
# 予想PER：株価を1株当たりの利益で割って求めます。PER（倍） ＝ 株価 ÷ 1株当たり当期純利益の通期予想
df_work["予想PER"] = df_work["AdjustmentClose"] / df_work["ForecastProfit"]

# # 配当性向を算出します
# df_work["配当性向"] = df_work["ResultDividendPerShareFiscalYearEnd"] / df_work["EarningsPerShare"] 
# # 予想配当性向を算出します
# df_work["予想配当性向"] = df_work["ForecastDividendPerShareAnnual"] / df_work["ForecastEarningsPerShare"]

# 銘柄名と結合します
df_work = pd.merge(df_work, df_list, left_on=["LocalCode"], right_on=["Code"])

# 表示用に開示日を追加します
df_work["開示日"] = df_work["DisclosedDate"].dt.strftime("%Y-%m-%d")
# グラフプロット時のX軸データを作成
# df_work["plot_x"] = int( df_work["DisclosedDate"].dt.strftime("%Y")) + (int( df_work["DisclosedDate"].dt.strftime("%m") ) / 12)



/var/folders/kx/n6w_n8193fn6dy4jwf3fsr6w0000gn/T/ipykernel_2726/570123662.py:24: FutureWarning: Passing 'suffixes' which cause duplicate columns {'AdjustmentClose_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_work = pd.merge(df_work, df_close, left_on=["LocalCode"], right_on=["Code"], how="left")


KeyError: 'AdjustmentClose'

In [25]:
 df_work["EarningsPerShare"]

0       -31.94
1       -22.23
2        33.22
3          NaN
4        48.11
         ...  
4314     -1.91
4315    220.99
4316     68.51
4317     31.72
4318       NaN
Name: EarningsPerShare, Length: 4319, dtype: float64

In [ ]:
# 表示する項目を指定します
output_cols = [
    "LocalCode",
    "CompanyName",
    "開示日",
    "配当性向",
    "予想配当性向",
    "配当利回り",
    "予想配当利回り",
    "1ヶ月リターン",
    "3ヶ月リターン",
    "PER"
    "予想PER"
]
# 項目別に表示方法を指定します
output_format = {
    "配当性向": "{:.2%}",
    "予想配当性向": "{:.2%}",
    "配当利回り": "{:.2%}",
    "予想配当利回り": "{:.2%}",
    "1ヶ月リターン": "{:.2%}",
    "3ヶ月リターン": "{:.2%}",
}

## データの表示

In [ ]:
# 配当利回り順に上位20銘柄を表示します
df_work.sort_values(["配当利回り"], ascending=False)[output_cols].head(20).style.format(output_format)

In [ ]:
# 予想配当利回り順に上位20銘柄を表示します
df_work.sort_values(["予想配当利回り"], ascending=False)[output_cols].head(20).style.format(output_format)

In [ ]:
# 横軸に配当利回り、縦軸に1,3ヶ月リターンとして関係性をプロットします
display("1ヶ月リターン")
sns.jointplot(x='配当利回り', y='1ヶ月リターン', data=df_work.loc[df_work["配当利回り"] > 0.00], kind="reg")
plt.show()

display("３ヶ月リターン")
sns.jointplot(x='配当利回り', y='3ヶ月リターン', data=df_work.loc[df_work["配当利回り"] > 0.00], kind="reg")
plt.show()

In [ ]:
# 横軸に予想配当利回り、縦軸に1,3,6ヶ月リターンとして関係性をプロットします
display("1ヶ月リターン")
sns.jointplot(x='予想配当利回り', y='1ヶ月リターン', data=df_work.loc[df_work["予想配当利回り"] > 0.00], kind="reg")
plt.show()

display("３ヶ月リターン")
sns.jointplot(x='予想配当利回り', y='3ヶ月リターン', data=df_work.loc[df_work["予想配当利回り"] > 0.00], kind="reg")
plt.show()

In [ ]:
# 予想配当利回りが5%以上の銘柄とリターンの関係性をプロットします
display("1ヶ月リターン")
sns.jointplot(x='予想配当利回り', y='1ヶ月リターン', data=df_work.loc[df_work["予想配当利回り"] >= 0.05], kind="reg")
plt.show()

display("３ヶ月リターン")
sns.jointplot(x='予想配当利回り', y='3ヶ月リターン', data=df_work.loc[df_work["予想配当利回り"] >= 0.05], kind="reg")
plt.show()

## 遊びデータ

In [ ]:
df_work

In [ ]:
# 作業用にデータをコピーします
df_work

In [ ]:
# 作業用にデータをコピーします
df_work = df_s.copy()
df_p_work = df_p.copy()

df_work[df_work["LocalCode"] == "72030" ]
# df_work[df_work["DisclosureNumber"] == "20180312488206" ]

# df_work[ [df_work["LocalCode"] == "72030"][df_work["DisclosureNumber"] == "20180312488206"] ]

# df_work[df_work["TypeOfCurrentPeriod" == "FY"]]



In [ ]:
# 作業用にデータをコピーします
df_work = df_s.copy()
df_p_work = df_p.copy()

x = df_work[df_work["LocalCode"] == "34870" ]['DisclosedDate'].to_list()
y = df_work[df_work["LocalCode"] == "34870" ]['DisclosedUnixTime'].to_list()
print(x,y)

plt.ion()

fig, ax = plt.subplots(1, 1)
ax.set_title('twin x')
ax.set_xlabel('x')
ax.set_ylabel('y1, y2')
ax.plot(x, y, 'o-', ms=4, color='red', label=f"y1")
# fig.savefig('twinx')

ax2 = ax.twinx()
ax2.set_ylabel('y1, y2')
x = df_work[df_work["LocalCode"] == "34870" ]['DisclosedDate'].to_list()
y = df_work[df_work["LocalCode"] == "34870" ]['ForecastNetSales'].to_list()
ax2.plot(x, y, 'o-', ms=4, color='black', label=f"y1")

# h1, l1 = ax.get_legend_handles_labels()
# h2, l2 = ax2.get_legend_handles_labels()
# ax.legend(h1 + h2, l1 + l2)

# グラフの保存
# fig.savefig('twinx')


In [ ]:
a = x[22938]
print(a)
a = int(a.strftime("%Y")) + int(a.strftime("%m")) / 12 
print(type(a))
print(a)

# a = int( a.strftime("%m") ) / 12
# print(a)

In [ ]:
from datetime import datetime, date

# 文字列（YYYY-MM-DD HH:MM:SS)
str_type      = '2022-01-01 18:05:00'

# 日付変換（YYYY-MM-DD HH:MM:SS)
datetime_type = datetime.strptime(str_type, '%Y-%m-%d %H:%M:%S')

# Datetime → Date変換（YYYY-MM-DD)
date_type     = date(datetime_type.year, datetime_type.month, datetime_type.day)

print(date_type)

# 出力イメージ（date_type）
# 2022-01-01


# 本日日付 （Datetime）
today_datetime_type = datetime.today()

# 文字列変換（YYYY-MM-DD)
today_str_type      = today_datetime_type.strftime("%Y-%m-%d")

print(today_str_type)
# 出力結果（today_str_type）
# 2022-12-17

df_s.loc[:, "CurrentFiscalYearStartDate"] = pd.to_datetime(
    df_s["CurrentFiscalYearStartDate"], format="%Y-%m-%d"
)

In [ ]:

x = df_work[df_work["LocalCode"] == "34870" ]['DisclosedDate']
x = x.to_datetime()
# .strftime("%Y")
print(x)

In [ ]:
# 作業用にデータをコピーします
df_work = df_s.copy()
df_p_work = df_p.copy()

x = df_work[df_work["LocalCode"] == "34870" ]['DisclosedDate'].to_list()
y = df_work[df_work["LocalCode"] == "34870" ]['DisclosedUnixTime'].to_list()
print(x,y)

plt.ion()

fig, ax = plt.subplots(1, 1)
ax.set_title('twin x')
ax.set_xlabel('x')
ax.set_ylabel('y1, y2')
ax.plot(x, y, 'o-', ms=4, color='red', label=f"y1")
# fig.savefig('twinx')


ax2 = ax.twinx()
ax2.set_ylabel('y1, y2')
a, b = np.polyfit(x, y, 1)
x = df_work[df_work["LocalCode"] == "34870" ]['DisclosedDate'].to_list()
y = a * x - b
ax2.plot(x, y, 'o-', ms=4, color='black', label=f"y1")

# h1, l1 = ax.get_legend_handles_labels()
# h2, l2 = ax2.get_legend_handles_labels()
# ax.legend(h1 + h2, l1 + l2)

# グラフの保存
# fig.savefig('twinx')


In [ ]:
df_p_work

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.ion()
x = np.arange(10)

fig, ax = plt.subplots(1, 1)
ax.set_title('twin x')
ax.set_xlabel('x')
ax.set_ylabel('y1, y2')
for order in range(1, 2 + 1):
    y = x ** order
    color = ['red', 'blue']
    ax.plot(x, y, 'o-', ms=4, color=color[order - 1], label=f"y{order}")

ax2 = ax.twinx()
ax2.set_ylabel('y3, y4')
for order in range(3, 4 + 1):
    y = x ** order
    color = ['green', 'orange']
    ax2.plot(x, y, 'o-', ms=4, color=color[order - 3], label=f"y{order}")

h1, l1 = ax.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()
ax.legend(h1 + h2, l1 + l2)

fig.savefig('twinx')